In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

import os

### **Buffer files merge**

**코호트 주소 정보 import**

In [4]:
coh = pd.read_csv('코호트_addresscsv', encoding='euc-kr')
coh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4776 entries, 0 to 4775
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       4776 non-null   object
 1   address  4776 non-null   object
dtypes: object(2)
memory usage: 74.8+ KB


**Buffer 500m**

In [9]:
bf500m = pd.read_csv('./2015년/2015_buffer500m/201501_mean_EVI_by_buffer500m.csv')
bf500m = bf500m.sort_values(by='ORIG_FID').reset_index(drop=True)

bf500m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4776 entries, 0 to 4775
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ORIG_FID  4776 non-null   int64  
 1   mean      1229 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 74.8 KB


In [17]:
def merge_buffers(path, cohort):
    import os
    
    buff_list = os.listdir(path)
    mean_list = sorted([c for c in buff_list if 'mean' in c])
    
    for index in ['EVI', 'NDVI']:
        
        merge_list = cohort.copy()
        
        for m in mean_list:
            buffer_size = m.split('_')[-1]
        
            s = [c for c in buff_list if 'std' in c and buffer_size in c][0]
        
            mean = pd.read_csv(path+'//'+m, encoding='euc-kr')
            std = pd.read_csv(path+'//'+s, encoding='euc-kr')
        
            if cohort.shape[0] != mean.shape[0]:
                raise Exception('코호트 파일과 buffer 파일 길이가 일치하지 않습니다.')
        
            mean = mean.sort_values(by='ORIG_FID').reset_index(drop=True)
            mean = mean.drop(columns='ORIG_FID')
        
            std = std.sort_values(by='ORIG_FID').reset_index(drop=True)
            std = std.drop(columns='ORIG_FID')
                
            merge = pd.concat([coh, mean, std], axis=1)
            merge = merge.loc[:, ['id', 'address', 'mean','stdDev']].rename(columns={'mean': f'{buffer_size}_avg',
                                                                                 'stdDev': f'{buffer_size}_std'})

        merge_list = pd.merge(merge_list, merge.drop(columns='address'), on='id', how='outer')
        
        if index == 'EVI':
            EVI = merge_list.copy()
        else: NDVI = merge_list.copy()
        
    fin_merge = pd.merge(EVI, NDVI.drop(columns='address'), on='id', how='outer')
    
    return fin_merge